# 영화 평점 분석 실습

In [1]:
# 라이브러리 불러오기
import pandas as pd

## 1. 영화 평점 데이터 적재 및 결합

In [2]:
# 사용자 정보 적재
users = pd.read_csv('movie_users.csv')
users.drop('Unnamed: 0', axis=1, inplace=True)
users.head()

,사용자아이디,성별,연령,직업,지역
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [3]:
# 영화 평점 정보 적재
ratings = pd.read_csv('movie_ratings.csv')
ratings.drop('Unnamed: 0', axis=1, inplace=True)
ratings.head()

,사용자아이디,영화아이디,평점,타임스탬프
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [4]:
# 영화 정보 적재
movies = pd.read_csv('movie_details.csv')
movies.drop('Unnamed: 0', axis=1, inplace=True)
movies.head()

,영화아이디,제목,장르
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
# 사용자수, 영화수, 평점수 개수 확인
print('''사용자수 : {}
영화수 : {}
평점수 : {}'''.format(len(users), len(movies), len(ratings)))

사용자수 : 6040
영화수 : 3883
평점수 : 1000209


### 1. 데이터 결합
#### 1-1) 사용자 데이터와 영화 평점 데이터를 사용자아이디 컬럼 기준으로 공통으로 존재하는 데이터만 결합

In [ ]:
users.columns

In [ ]:
ratings.columns

In [ ]:
users.shape

In [ ]:
ratings.shape

In [6]:
# 평점을 남긴 사용자의 아이디는 users에 모두 존재하는 아이디
data1 = pd.merge(users, ratings)
data1.shape

(1000209, 8)

#### 1-2) 1번 결과에 영화 아이디 컬럼 기준으로 영화 정보 추가하기

In [ ]:
data1.head()

In [ ]:
movies.head()

In [7]:
data2 = pd.merge(data1, movies)
data2.head()

,사용자아이디,성별,연령,직업,지역,영화아이디,평점,타임스탬프,제목,장르
0,1,F,1,10,48067,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,M,56,16,70072,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,M,25,12,32793,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,M,25,7,22903,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,M,50,1,95350,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama


In [8]:
# data1에 있던 영화 아이디는 모두 movies의 영화 아이디에 포함
data2.shape

(1000209, 10)

### 2. 인기 있는 영화 10개 찾기

- 기준 : 평점이 4.5 이상이고 평점 개수가 2000개 이상인 영화

In [ ]:
# 평점이 높은 순서로 정렬
data2.sort_values(by='평점', ascending=False).head(10)

In [ ]:
# 하나의 영화에 여러개의 평점이 존재하는 1 : N 관계 => 평균
# 영화아이디별 평균 평점
# 로우 : 아이디/ 제목
# 값 : 평점
# 집계함수 : 평균
data2.pivot_table(index='제목', values='평점', aggfunc='mean').head()

In [ ]:
# 평균 평점을 기준으로 내림차순 정렬
data2.pivot_table(index='제목', values='평점', aggfunc='mean').sort_values(by='평점', ascending=False).head()

In [ ]:
# 평점 - 평균, 개수
# 로우 : 제목
# 값 : 평점
# 집계함수 : mean, count
r1 = data2.pivot_table(index='제목', values='평점', aggfunc=['mean', 'count'])

In [ ]:
# 평균 평점, 평점 개수
# 평점기준으로 내림차순 정렬
r1.head()
r1.columns

In [ ]:
# 하위 계층인 평점에 대해서 정렬하기
# 계층 색인을 모두 전달 => 튜플, 순서 맞춰서 전달
r1.sort_values(by=('mean','평점'), ascending=False).head()

In [ ]:
# r1에 대해 컬럼의 계층색인 재배열
# 평점 -> 상위, mean & count -> 하위
r2 = r1.swaplevel(1, 0, axis=1)

In [ ]:
# 평균평점이 동일할 경우, 평점개수가 많은 순서로 정렬
r2.sort_values(by=[('평점', 'mean'),('평점', 'count')], ascending=[False, False]).head()

In [ ]:
# 평점평균이 4.5 이상이고 평점 개수가 2000개 이상인 영화
# r2[('평점', 'mean')]
result = r2[(r2['평점']['mean'] >= 4.5) & (r2['평점']['count'] >= 2000)]

In [ ]:
result.index

### ▼ 이어서 진행
### 3. 여자들이 좋아하는 영화의 장르 찾기 

- 기준 : 여성 평균 평점이 4.0 이상이고 여성 평점의 개수가 500개 이상인 영화
- 최종 결과 : 평점 기준 상위 10위 장르 정보

In [9]:
data2.columns

Index(['사용자아이디', '성별', '연령', '직업', '지역', '영화아이디', '평점', '타임스탬프', '제목', '장르'], dtype='object')

In [13]:
female_data = data2[data2['성별'] == 'F'].copy()

In [15]:
data2.shape

(1000209, 10)

In [17]:
female_data.shape[0] / data2.shape[0]

0.24638850480249627

In [20]:
# 평점의 평균이 4.0 이상, 개수가 500개 이상
# 기준값 : 영화당 평점의 평균, 개수
tmp1 = female_data.pivot_table(index='제목', values='평점',
                        aggfunc=['mean', 'count'])

In [26]:
female_data[female_data['제목'] == '$1,000,000 Duck (1971)'].shape

(16, 10)

In [27]:
female_data[female_data['제목'] == '$1,000,000 Duck (1971)']['평점'].mean()

3.375

In [23]:
# 컬럼 계층 변경 : 보기 편한 구조로 바꾸기 위함
tmp1 = tmp1.swaplevel(1,0,axis=1)

In [31]:
# 제목으로 index를 지정한 경우
# 제목 값을 기준으로 장르 값을 찾아야한다.
female_movies = tmp1[(tmp1[('평점', 'mean')] >= 4.0) & (tmp1[('평점','count')] >=500)].index
female_movie_df = tmp1[(tmp1[('평점', 'mean')] >= 4.0) & (tmp1[('평점','count')] >=500)].copy()

In [32]:
movies.head()

,영화아이디,제목,장르
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [35]:
# 공통 컬럼으로 결합 => merge
tmp = female_movie_df.reset_index()

In [40]:
tmp.merge(movies, on='제목')[['제목', '장르']].head()

,제목,장르
0,American Beauty (1999),Comedy|Drama
1,Being John Malkovich (1999),Comedy
2,Braveheart (1995),Action|Drama|War
3,Casablanca (1942),Drama|Romance|War
4,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi


In [42]:
# 처음부터 장르값 가져오기
tmp2 = female_data.pivot_table(index='장르', values='평점',
                       aggfunc=['mean','count'])

In [56]:
tmp2[(tmp2[('mean','평점')]>=4.0) & (tmp2[('count','평점')]>=500)].shape

(9, 2)

In [48]:
# 기준을 만족하는 영화의 제목과 장르를 한번에 추출
tmp3 = female_data.pivot_table(index=['제목','장르'], values='평점',
                       aggfunc=['mean','count'])

In [51]:
# 기준을 만족하는 영화의 제목과 장르를 한번에 추출
tmp3[(tmp3[('mean','평점')]>=4.0) & (tmp3[('count','평점')]>=500)].index.values

array([('American Beauty (1999)', 'Comedy|Drama'),
       ('Being John Malkovich (1999)', 'Comedy'),
       ('Braveheart (1995)', 'Action|Drama|War'),
       ('Casablanca (1942)', 'Drama|Romance|War'),
       ('E.T. the Extra-Terrestrial (1982)', "Children's|Drama|Fantasy|Sci-Fi"),
       ('Fargo (1996)', 'Crime|Drama|Thriller'),
       ('Forrest Gump (1994)', 'Comedy|Romance|War'),
       ('L.A. Confidential (1997)', 'Crime|Film-Noir|Mystery|Thriller'),
       ('Matrix, The (1999)', 'Action|Sci-Fi|Thriller'),
       ('Princess Bride, The (1987)', 'Action|Adventure|Comedy|Romance'),
       ('Pulp Fiction (1994)', 'Crime|Drama'),
       ('Raiders of the Lost Ark (1981)', 'Action|Adventure'),
       ('Saving Private Ryan (1998)', 'Action|Drama|War'),
       ("Schindler's List (1993)", 'Drama|War'),
       ('Shakespeare in Love (1998)', 'Comedy|Romance'),
       ('Shawshank Redemption, The (1994)', 'Drama'),
       ('Silence of the Lambs, The (1991)', 'Drama|Thriller'),
       ('Sixth Sen

In [55]:
tmp3.shape[0]

3481

In [57]:
tmp3.head(2)

,,mean,count
,,평점,평점
제목,장르,,
"$1,000,000 Duck (1971)",Children's|Comedy,3.375000,16
'Night Mother (1986),Drama,3.388889,36


In [54]:
'Children\'s'

"Children's"

### 장르 데이터 전처리

- 세부장르 분석을 위해 문자열에 대한 전처리 수행

In [66]:
# 중복을 제거한 장르 고유값
# set() 타입을 활용한 고유값 처리
dump = []
for i in tmp2.index.values:
    tmp = i.split('|')
    for j in tmp:
        dump.append(j)

In [84]:
len(set(dump))

18

In [72]:
result = tmp2.reset_index().copy()

In [76]:
female_fav_genre = result['장르'].str.split('|', expand=True)

In [77]:
female_fav_genre.values

array([['Action', None, None, None, None, None],
       ['Action', 'Adventure', None, None, None, None],
       ['Action', 'Adventure', 'Animation', None, None, None],
       ...,
       ['Thriller', None, None, None, None, None],
       ['War', None, None, None, None, None],
       ['Western', None, None, None, None, None]], dtype=object)

In [81]:
import numpy as np

np.unique(female_fav_genre.fillna('-'))[1:]

array(['Action', 'Adventure', 'Animation', "Children's", 'Comedy',
       'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
       'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War',
       'Western'], dtype=object)

In [83]:
female_fav_genre.head(2)

,0,1,2,3,4,5
0,Action,None,None,None,None,None
1,Action,Adventure,None,None,None,None


In [ ]:
# 가장 인기 있는 세부장르 top3
# 장르 빈도

In [86]:
female_fav_genre.head(3)

,0,1,2,3,4,5
0,Action,None,None,None,None,None
1,Action,Adventure,None,None,None,None
2,Action,Adventure,Animation,None,None,None


In [88]:
# 시리즈 간의 연산
# sr.value_counts()
female_fav_genre.count()

0    300
1    283
2    203
3     80
4     14
5      1
dtype: int64

In [90]:
data2['장르']

0                           Drama
1                           Drama
2                           Drama
3                           Drama
4                           Drama
5                           Drama
6                           Drama
7                           Drama
8                           Drama
9                           Drama
10                          Drama
11                          Drama
12                          Drama
13                          Drama
14                          Drama
15                          Drama
16                          Drama
17                          Drama
18                          Drama
19                          Drama
20                          Drama
21                          Drama
22                          Drama
23                          Drama
24                          Drama
25                          Drama
26                          Drama
27                          Drama
28                          Drama
29            

In [93]:
cat1 = female_fav_genre[0].value_counts()
cat2 = female_fav_genre[1].value_counts()
cat3 = female_fav_genre[2].value_counts()
cat4 = female_fav_genre[3].value_counts()
cat5 = female_fav_genre[4].value_counts()
cat6 = female_fav_genre[5].value_counts()

In [95]:
cat5

Thriller    3
War         3
Sci-Fi      3
Fantasy     2
Romance     2
Musical     1
Name: 4, dtype: int64

In [96]:
cat1 + cat2 + cat3 + cat4 + cat5 + cat6

Action          NaN
Adventure       NaN
Animation       NaN
Children's      NaN
Comedy          NaN
Crime           NaN
Documentary     NaN
Drama           NaN
Fantasy         NaN
Film-Noir       NaN
Horror          NaN
Musical         NaN
Mystery         NaN
Romance         NaN
Sci-Fi          NaN
Thriller        NaN
War            34.0
Western         NaN
dtype: float64

In [100]:
result = cat1.add(cat2, fill_value=0)
result = result.add(cat3,fill_value=0)
result = result.add(cat4,fill_value=0)
result = result.add(cat5,fill_value=0)
result = result.add(cat6,fill_value=0)

In [103]:
result.sort_values(ascending=False).head(3)

Action       96.0
Adventure    86.0
Comedy       86.0
dtype: float64

In [106]:
# 함수
data = np.unique(female_fav_genre.fillna('-'), return_counts=True)

In [108]:
data[0]

array(['-', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy',
       'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
       'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War',
       'Western'], dtype=object)

In [107]:
data[1]

array([919,  96,  86,  29,  55,  86,  42,   5,  78,  33,  18,  30,  31,
        36,  59,  69,  77,  34,  17], dtype=int64)

In [111]:
result = pd.Series(data[1][1:], index=data[0][1:])
result

Action         96
Adventure      86
Animation      29
Children's     55
Comedy         86
Crime          42
Documentary     5
Drama          78
Fantasy        33
Film-Noir      18
Horror         30
Musical        31
Mystery        36
Romance        59
Sci-Fi         69
Thriller       77
War            34
Western        17
dtype: int64

### 4. 연령대 별 영화 평점 분석하기

- 사용자정의함수를 사용하여 처리

- 50대가 가장 좋아하는 영화 top 10
    - 평균평점이 높은 순서로 상위 10개의 제목

In [112]:
data2['연령']

0           1
1          56
2          25
3          25
4          50
5          18
6           1
7          25
8          25
9          45
10         18
11         25
12         45
13         18
14         25
15         18
16         25
17         50
18         25
19         50
20         35
21         56
22         25
23         45
24         56
25         45
26         25
27          1
28         35
29         25
           ..
1000179    25
1000180    56
1000181    18
1000182    25
1000183    45
1000184    45
1000185    25
1000186    18
1000187    50
1000188    35
1000189    25
1000190    25
1000191    25
1000192    18
1000193    18
1000194    25
1000195    56
1000196    25
1000197    56
1000198    56
1000199    56
1000200     1
1000201    35
1000202    35
1000203    45
1000204    18
1000205    35
1000206    18
1000207    18
1000208    25
Name: 연령, Length: 1000209, dtype: int64

In [ ]:
# 10s, 20s, 30s, 40s, 50s, 60s

In [ ]:
# 연령대
# 연속형 변수를 범주형 변수
# 1. 사용자정의함수
# 2. pd.cut()
# 3. np.digitize()

In [115]:
data2['연령'].describe().astype(int)

count    1000209
mean          29
std           11
min            1
25%           25
50%           25
75%           35
max           56
Name: 연령, dtype: int32

In [116]:
# 1. 사용자정의함수
# 0~19 : 10대
# 20~29 : 20대
# 30~39 : 30대
# 40~49 : 40대
# 50~59 : 50대
def age_range(x):
    if x < 20:
        label = '10대'
    elif x < 30:  # 20 <= x < 30
        label = '20대'
    elif x < 40:  # 30 <= x < 40
        label = '30대'
    elif x < 50:  
        label = '40대'
    else:    # x >= 50
        label = '50대'
        
    return label

In [118]:
data2['연령1'] = data2['연령'].apply(age_range)

In [123]:
data2[data2['연령'] < 20]['연령1'].unique()
data2[(20 < data2['연령']) & (data2['연령'] < 30)]['연령1'].unique()

array(['20대'], dtype=object)

In [125]:
# pd.cut()
pd.cut([1,2,3,4,5,6], [0,2,4,6])

[(0, 2], (0, 2], (2, 4], (2, 4], (4, 6], (4, 6]]
Categories (3, interval[int64]): [(0, 2] < (2, 4] < (4, 6]]

In [128]:
data2['연령2'] = pd.cut(data2['연령'], [0,20,30,40,50,60],
       labels=['10대', '20대', '30대', '40대', '50대'])

In [131]:
# np.digitize()
# 0 : 10대
# 4 : 50대
data2['연령3'] = np.digitize(data2['연령'], [20,30,40,50])

In [132]:
data2['연령3'].head()

0    0
1    4
2    1
3    1
4    4
Name: 연령3, dtype: int64

In [134]:
# 매핑처리 필요
# 딕셔너리
# 0~4
my_dict = {}
for i,j in enumerate(['10대', '20대', '30대', '40대', '50대']):
    my_dict[i] = j

In [135]:
my_dict

{0: '10대', 1: '20대', 2: '30대', 3: '40대', 4: '50대'}

In [139]:
data2['연령3'] = data2['연령3'].map(my_dict)

In [140]:
data2[['연령','연령3']].head()

,연령,연령3
0,1,10대
1,56,50대
2,25,20대
3,25,20대
4,50,50대


In [ ]:
# 50대 인기 영화 제목
# 인기 : 평균 평점이 높은 순서로 상위 top 10 영화 제목

In [143]:
# 50대 추출
age50_data = data2[data2['연령1']=='50대'][['연령1', '평점', '제목']].copy()

In [144]:
age50_data.head()

,연령1,평점,제목
1,50대,5,One Flew Over the Cuckoo's Nest (1975)
4,50대,5,One Flew Over the Cuckoo's Nest (1975)
17,50대,5,One Flew Over the Cuckoo's Nest (1975)
19,50대,4,One Flew Over the Cuckoo's Nest (1975)
21,50대,4,One Flew Over the Cuckoo's Nest (1975)


In [147]:
# 평균평점순으로 상위 10개
tmp = age50_data.pivot_table(index='제목', values='평점', aggfunc='mean')

In [151]:
tmp.sort_values(by='평점', ascending=False).head(10).index

Index(['It Takes Two (1995)', 'Paris, France (1993)',
       'Tetsuo II: Body Hammer (1992)',
       'Schlafes Bruder (Brother of Sleep) (1995)',
       'My Life and Times With Antonin Artaud (En compagnie d'Antonin Artaud) (1993)',
       'Lured (1947)', 'Ed's Next Move (1996)', 'Smashing Time (1967)',
       'Apple, The (Sib) (1998)', 'Love Serenade (1996)'],
      dtype='object', name='제목')

In [157]:
tmp2 = data2.pivot_table(index='제목', values='평점', columns='연령1',
                 aggfunc='mean', fill_value=-9999)

In [158]:
tmp2.isnull().sum()

연령1
10대    0
20대    0
30대    0
40대    0
50대    0
dtype: int64

In [165]:
tmp2['50대'].sort_values(ascending=False).index[:10]

Index(['Better Living Through Circuitry (1999)',
       'Ballad of Narayama, The (Narayama Bushiko) (1958)',
       'Gambler, The (A J�t�kos) (1997)',
       'Mating Habits of the Earthbound Human, The (1998)',
       'Tetsuo II: Body Hammer (1992)', 'Lured (1947)',
       'Identification of a Woman (Identificazione di una donna) (1982)',
       'Across the Sea of Time (1995)', 'Love Serenade (1996)',
       'Terrorist, The (Malli) (1998)'],
      dtype='object', name='제목')

### 5. 남자와 여자의 호불호가 크게 갈리는 영화 10개를 출력 
#### 전체 평점의 개수가 500개 이상인 영화만 대상으로 함.

- 호불호 : 성별 평점의 차이의 크기가 크다 => 새로운 컬럼으로 저장
- 성별 평점 차이의 크기 기준으로 내림차순 정렬 하여 상위 10개 영화의 제목 찾기

In [168]:
# 전체 평점의 개수가 500개 이상인 영화
tmp = data2.pivot_table(index='제목', values='평점',
                        aggfunc='count')

In [171]:
movies = tmp[tmp['평점'] >= 500].index

In [172]:
data2.shape

(1000209, 13)

In [175]:
# 제목 컬럼
# 1. sr.isin([])
movies = movies.tolist()

In [178]:
final1 = data2[data2['제목'].isin(movies)]

In [182]:
final1.shape

(594712, 13)

In [181]:
# 2. sr.apply(lambda x: x in [])
data2[data2['제목'].apply(lambda x: x in movies)].shape

(594712, 13)

In [184]:
final1.shape

(594712, 13)

In [183]:
final1.head()

,사용자아이디,성별,연령,직업,지역,영화아이디,평점,타임스탬프,제목,장르,연령1,연령2,연령3
0,1,F,1,10,48067,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,10대,10대,10대
1,2,M,56,16,70072,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama,50대,50대,50대
2,12,M,25,12,32793,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama,20대,20대,20대
3,15,M,25,7,22903,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama,20대,20대,20대
4,17,M,50,1,95350,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama,50대,40대,50대


In [193]:
df = data2.pivot_table(index=['제목','성별'], values='평점',
                  aggfunc=['mean', 'count']).unstack()

In [196]:
tmp = df[('count','평점')]

In [198]:
tmp['total'] = tmp['F'] + tmp['M']

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [201]:
tmp[tmp['total'] >= 500].index

Index(['10 Things I Hate About You (1999)', '101 Dalmatians (1961)',
       '12 Angry Men (1957)', '13th Warrior, The (1999)',
       '20,000 Leagues Under the Sea (1954)', '2001: A Space Odyssey (1968)',
       '28 Days (2000)', 'Abyss, The (1989)',
       'Ace Ventura: Pet Detective (1994)', 'Addams Family, The (1991)',
       ...
       'Witness (1985)', 'Wizard of Oz, The (1939)', 'Working Girl (1988)',
       'World Is Not Enough, The (1999)', 'Wrong Trousers, The (1993)',
       'X-Files: Fight the Future, The (1998)', 'X-Men (2000)',
       'You've Got Mail (1998)', 'Young Frankenstein (1974)',
       'Young Guns (1988)'],
      dtype='object', name='제목', length=618)

In [203]:
final1.shape

(594712, 13)

In [205]:
final1.pivot_table(index=['제목', '성별'], values='평점',
                  aggfunc='mean').unstack()

평점          
성별                                                         F         M
제목                                                                    
10 Things I Hate About You (1999)                   3.646552  3.311966
101 Dalmatians (1961)                               3.791444  3.500000
12 Angry Men (1957)                                 4.184397  4.328421
13th Warrior, The (1999)                            3.112000  3.168000
20,000 Leagues Under the Sea (1954)                 3.670103  3.709205
2001: A Space Odyssey (1968)                        3.825581  4.129738
28 Days (2000)                                      3.209424  2.977707
Abyss, The (1989)                                   3.659236  3.689507
Ace Ventura: Pet Detective (1994)                   3.000000  3.197917
Addams Family, The (1991)                           3.186170  3.163498
Adventures in Babysitting (1987)                    3.455782  3.208122
Adventures of Buckaroo Bonzai Across the 8th Di...  3.308511  3.402321
African Queen, The (1951)                           4.324232  4.223822
Air Force One (1997)                                3.699588  3.555822
Airplane II: The Sequel (1982)                      2.906250  2.995671
Airplane! (1980)                                    3.656566  4.064419
Aladdin (1992)                                      3.857143  3.756494
Alice in Wonderland (1951)                          3.705882  3.692308
Alien (1979)                                        3.888252  4.216119
Alien Nation (1988)                                 3.433333  3.195946
Alien: Resurrection (1997)                          2.708738  2.997041
Aliens (1986)                                       3.802083  4.186684
Alien� (1992)                                       3.008264  3.042289
Almost Famous (2000)                                4.220217  4.228731
Amadeus (1984)                                      4.346734  4.213415
American Beauty (1999)                              4.238901  4.347301
American Graffiti (1973)                            3.940476  4.055556
American History X (1998)                           4.181818  4.240741
American Pie (1999)                                 3.539792  3.754545
American President, The (1995)                      3.923483  3.718654
...                                                      ...       ...
Wag the Dog (1997)                                  3.573222  3.459581
Waking Ned Devine (1998)                            3.963964  3.839917
War of the Worlds, The (1953)                       3.885246  3.855159
Waterworld (1995)                                   2.593496  2.640152
Wayne's World (1992)                                3.464286  3.646429
Wedding Singer, The (1998)                          3.581081  3.445351
Weird Science (1985)                                3.375000  3.435961
West Side Story (1961)                              4.228374  3.953390
Westworld (1973)                                    3.371795  3.428315
What About Bob? (1991)                              3.429293  3.387722
What's Eating Gilbert Grape (1993)                  3.879668  3.820796
When Harry Met Sally... (1989)                      4.257028  3.987850
While You Were Sleeping (1995)                      3.636066  3.385870
White Men Can't Jump (1992)                         3.028777  3.231061
Who Framed Roger Rabbit? (1988)                     3.569378  3.713251
Whole Nine Yards, The (2000)                        3.296552  3.404814
Wild Things (1998)                                  3.392000  3.459082
Wild Wild West (1999)                               2.275449  2.131973
Willow (1988)                                       3.658683  3.453543
Willy Wonka and the Chocolate Factory (1971)        4.063953  3.789474
Witness (1985)                                      4.115854  3.941504
Wizard of Oz, The (1939)                            4.355030  4.203138
Working Girl (1988)                                 3.606742  3.312500
World Is No

In [207]:
# columns
pivot1 = final1.pivot_table(index='제목', values='평점', columns='성별',
                  aggfunc='mean')

In [213]:
# 절대값 : abs(), np.abs()
pivot1['남녀차이'] = np.abs(pivot1['F'] - pivot1['M'])

In [ ]:
pivot1['F'].sub(picot1['M'])

In [220]:
pivot1['남녀차이'].sort_values(ascending=False).head(10).index.tolist()

['Dirty Dancing (1987)',
 'Good, The Bad and The Ugly, The (1966)',
 'Dumb & Dumber (1994)',
 'Evil Dead II (Dead By Dawn) (1987)',
 'Grease (1978)',
 'Caddyshack (1980)',
 'Animal House (1978)',
 'Exorcist, The (1973)',
 'Rocky Horror Picture Show, The (1975)',
 'Big Trouble in Little China (1986)']